## Prediction with Style-color-size dataset - Multi label classification 

I have made the database of photos sorted by products and brands. Screenshots were performed only on official brand websites.

The main dataset (style.zip) is 2184 color images (150x150x3) with 7 brands and 10 products, and the file with labels style.csv.
Photo files are in the .png format and the labels are integers and values.

Above para from - https://www.kaggle.com/olgabelitskaya/style-color-images  

kaggle datasets download -d olgabelitskaya/style-color-images 

https://github.com/Kaggle/kaggle-api - reference for kaggle cli

In [0]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [0]:
# Set up fastai for collab 
!curl -s https://course.fast.ai/setup/colab | bash
!pip uninstall torch torchvision -y
!pip install torch==1.4.0 torchvision==0.5.0

In [0]:
#from google.colab import drive
#drive.mount('/content/gdrive', force_remount=True)
#root_dir = "/content/gdrive/My Drive/"
#base_dir = root_dir + 'fastai-v3/'

In [0]:
from fastai.vision import *

## Getting the data

First, install the Kaggle API by uncommenting the following line and executing it.

In [0]:
! pip install -q kaggle  

Then you need to upload your credentials from Kaggle on your instance. Login to kaggle and click on your profile picture on the top left corner, then 'My account'. Scroll down until you find a button named 'Create New API Token' and click on it. This will trigger the download of a file named 'kaggle.json'.

Upload this file to the directory this notebook is running in, by clicking "Upload" on your main Jupyter page, then uncomment and execute the next two commands (or run them in a terminal). For Windows, uncomment the last two commands.

In [0]:
from google.colab import files

files.upload()

In [0]:
! mkdir -p ~/.kaggle/
! mv kaggle.json ~/.kaggle/

# For Windows, uncomment these two commands
# ! mkdir %userprofile%\.kaggle
# ! move kaggle.json %userprofile%\.kaggle

In [0]:
! chmod 600 ~/.kaggle/kaggle.json
# ! kaggle datasets list

In [0]:
path = Config.data_path()/'style-color'
path.mkdir(parents=True, exist_ok=True)
path

In [0]:
! kaggle datasets download -d olgabelitskaya/style-color-images --force -p {path} --unzip 

In [0]:
path.ls()
# Training data 
path_style = path/'style'
# path_style.ls()

## Image multi label classification

In [0]:
temp = pd.read_csv(path_style/'style.csv')
temp['file'] = 'style/' + temp['file'].astype(str)
# temp.head() 
df = temp[['file', 'brand_label', 'product_label']]
df.head()

In [0]:
bs=64
tfms = get_transforms(do_flip=False) 

In [0]:
np.random.seed(42)
src = (ImageList.from_df(df, path)
       .split_by_rand_pct(0.2)
       .label_from_df(cols=['brand_label', 'product_label']))

In [0]:
data = (src.transform(tfms, size=128)
        .databunch().normalize(imagenet_stats))

In [0]:
# help(data.train_ds)
# df.iloc[536, :]


In [0]:
data.show_batch(rows=3, figsize=(12,9))

To create a `Learner` we use the same function as in lesson 1. Our base architecture is resnet50 again, but the metrics are a little bit differeent: we use `accuracy_thresh` instead of `accuracy`. In lesson 1, we determined the predicition for a given class by picking the final activation that was the biggest, but here, each activation can be 0. or 1. `accuracy_thresh` selects the ones that are above a certain threshold (0.5 by default) and compares them to the ground truth.

As for Fbeta, it's the metric that was used by Kaggle on this competition. See [here](https://en.wikipedia.org/wiki/F1_score) for more details.

In [0]:
arch = models.resnet50

In [0]:
# acc_02 = partial(accuracy_thresh, thresh=0.2)
# f_score = partial(fbeta, thresh=0.2)
# learn = cnn_learner(data, arch, metrics=[acc_02, f_score])
learn = cnn_learner(data, arch, metrics=accuracy)

We use the LR Finder to pick a good learning rate.

In [0]:
learn.lr_find()

In [0]:
learn.recorder.plot()

Then we can fit the head of our network.

In [0]:
lr = 1e-5

In [0]:
learn.fit_one_cycle(5, slice(lr))

In [0]:
learn.save('stage-1-rn50')

...And fine-tune the whole model:

In [0]:
learn.unfreeze()

In [0]:
learn.lr_find()
learn.recorder.plot()

In [0]:
learn.fit_one_cycle(5, slice(1e-5, lr/5))

In [0]:
learn.save('stage-2-rn50')

In [0]:
learn.recorder.plot_losses()

In [0]:
learn.export()